In [ ]:
# | default_exp client

In [ ]:
# | export

import inspect
import types
from typing import *

import airt
import typer
from airt._constant import *

In [ ]:
from airt._sanitizer import sanitized_print

In [ ]:
# | exporti


def _replace_env_var(s: str) -> str:
    return s.replace("AIRT_", "CAPTN_").replace("airt", "captn")

In [ ]:
# | exporti


def _fix_doc_string(cls: Type):
    cls.__doc__ = _replace_env_var(cls.__doc__)  # type: ignore

    for i in dir(cls):
        attr = getattr(cls, i)

        if (attr) and (not i.startswith("_")):
            for a in [attr, inspect.unwrap(attr)]:
                try:
                    if a.__class__ == types.MethodType:
                        a.__func__.__doc__ = _replace_env_var(a.__func__.__doc__)
                    else:
                        a.__doc__ = _replace_env_var(a.__doc__)

                except Exception as e:  # nosec B110
                    pass

                for ref in ["AIRT_", "airt"]:
                    if ref in a.__doc__:
                        raise ValueError(f"{ref} reference is available in {a.__doc__}")

In [ ]:
from contextlib import contextmanager

from fastcore.utils import patch


class A:
    """A"""

    pass


@patch(cls_method=True)
@contextmanager
def f(cls: A):
    """AIRT_"""
    pass


_fix_doc_string(A)

sanitized_print(A.f.__doc__)

assert A.f.__doc__ == "CAPTN_"

CAPTN_


In [ ]:
from airt.client import DataBlob

_fix_doc_string(DataBlob)

sanitized_print(DataBlob.set_default_cloud_provider.__doc__)

assert "AIRT_" not in DataBlob.set_default_cloud_provider.__doc__

Sets the default destination value for the cloud_provider and the region.

    Whenever you call the from_\* methods of the `DataBlob` class inside this context manager, the destination cloud_provider and region set in this context
    will be passed to the from_\* methods, unless you explicitely override it in the parameter.

    Args:
        cloud_provider: The destination cloud storage provider's name to store the datablob. Currently, the API only supports **aws** and **azure** as cloud storage providers.
        region: The destination cloud provider's region to store the datablob. The supported AWS regions are: ap-northeast-1, ap-northeast-2, ap-south-1, ap-southeast-1,
            ap-southeast-2, ca-central-1, eu-central-1, eu-north-1, eu-west-1, eu-west-2, eu-west-3, sa-east-1, us-east-1, us-east-2, us-west-1, us-west-2. The supported
            Azure Blob Storage regions are: australiacentral, australiacentral2, australiaeast, australiasoutheast, brazilsouth, canadacentral, c

In [ ]:
# | exporti


def _fix_cli_doc_string(app: typer.Typer, m_name: str):
    for c in app.registered_commands:
        name = c.callback.__name__.replace("-", "_")  # type: ignore

        getattr(getattr(airt._cli, m_name), name).__doc__ = _replace_env_var(
            getattr(getattr(airt._cli, m_name), name).__doc__
        )

In [ ]:
# | exporti


def _set_global_env_vars(m: types.ModuleType):
    for v in dir(m):
        if not v.startswith("_"):
            setattr(m, v, _replace_env_var(getattr(m, v)))

            if "AIRT_" in getattr(m, v):
                raise ValueError(f"AIRT_ reference is available in {getattr(m, v)}")

In [ ]:
# | export

_set_global_env_vars(airt._constant)

from airt.client import Client as _Client

Client = _Client

from airt.client import DataBlob as _DataBlob

DataBlob = _DataBlob

from airt.client import DataSource as _DataSource

DataSource = _DataSource

from airt.client import User as _User

User = _User

from airt.client import APIKey as _APIKey

APIKey = _APIKey

for cls in [Client, DataBlob, DataSource, User, APIKey]:
    cls.__module__ = "captn.client"
    _fix_doc_string(cls)